# MMA Flow

![alt](../../media/imgs/diag_flow_v2.png)

In [1]:
import os

GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")
if not GEMINI_API_KEY:
    raise EnvironmentError("GEMINI_API_KEY not found in environment variables. Please set it before running this notebook.")

In [2]:
from google import genai
import time

client = genai.Client(api_key=GEMINI_API_KEY)

In [4]:
TEST_VIDEO_FILE = "../../media/splits/holloway_gaethje.mp4"#"../../media/splits/round_chimaev_part_5.mp4"
assert os.path.exists(TEST_VIDEO_FILE), f"Media file {TEST_VIDEO_FILE} does not exist."


In [5]:
def upload_and_wait_for_file_processing(filepath, client, poll_interval=10, max_retries=10):
    uploaded_file = client.files.upload(file=filepath)
    retries = 0
    while uploaded_file.state.name == "PROCESSING":
        print('.', end='', flush=True)
        time.sleep(poll_interval)
        retries += 1
        if retries > max_retries:
            raise TimeoutError("El procesamiento del archivo tomó demasiado tiempo.")
        uploaded_file = client.files.get(name=uploaded_file.name)
    if uploaded_file.state.name == "FAILED":
        raise ValueError(f"Procesamiento fallido: {uploaded_file.state.name}")
    return uploaded_file

myfile = upload_and_wait_for_file_processing(
    filepath=TEST_VIDEO_FILE, # Replace here with slice video file path
    client=client,
    poll_interval=10,
    max_retries=10
)


..

In [6]:
def generate_general_analyst_prompt():
    """
    Genera un prompt para el Analista General que crea el análisis segundo a segundo
    (ground truth) que luego será usado por los 4 especialistas.
    """
    prompt = """
    **ROL Y OBJETIVO:**
    Eres un **General Analyst** trabajando bajo el **Head Coach de MMA**. Tu tarea es crear un análisis objetivo y completo segundo a segundo de la pelea, identificando qué está ocurriendo en cada momento y cuál es la disciplina más relevante. Este análisis servirá como **ground truth** (verdad fundamental) que será utilizado posteriormente por especialistas en diferentes áreas.

    **TU FUNCIÓN:**
    * Observar y documentar TODAS las acciones relevantes que ocurren en la pelea
    * Identificar la **disciplina más relevante** en cada segundo entre:
        * **Striking** (intercambio de golpes de pie: boxeo, muay thai, patadas)
        * **Grappling** (clinch, controles, derribos, trabajo de piso)
        * **Submission** (intentos de sumisión, transiciones, escapes)
        * **Movement** (footwork, posicionamiento, manejo de distancia, ángulos)
    * Crear un registro cronológico preciso que sirva como base para análisis especializados posteriores

    **INSTRUCCIONES CLAVE PARA LA TABLA:**
    * Genera una tabla de Markdown con **CUATRO columnas**: **Tiempo (MM:SS)**, **Peleador**, **Acción**, y **Disciplina Relevante**.
    * **IMPORTANTE - SAMPLING CADA 1 SEGUNDO:** Debes registrar una entrada en la tabla **cada 1 segundo** del video (00:00, 00:01, 00:02, 00:03, etc.), independientemente de si ocurre una acción significativa o no.
    * **Columna Tiempo (MM:SS):** Marca de tiempo en formato **Minuto:Segundo** (ej. `00:00`, `00:01`, `00:02`), alineado a intervalos de 1 segundo.
    * **Columna Peleador:** Nombre del Peleador A, Peleador B, o "Ambos" según corresponda.
    * **Columna Acción:** Descripción concisa y objetiva de lo que está ocurriendo en ese segundo. Debe ser descriptiva y neutral, sin interpretación especializada. Ejemplos:
        * Si hay golpes: "Jab conectado", "Cross conectado", "Patada a pierna"
        * Si hay grappling: "Takedown conseguido", "Control en clinch", "Pase de guardia"
        * Si hay sumisión: "Intento de estrangulación", "Escape de sumisión"
        * Si hay movimiento: "Cierre de distancia", "Ajuste de ángulo", "Footwork defensivo"
        * Si no hay acción relevante: "Intercambio de distancia", "Posicionamiento", "Evaluación", "Sin actividad relevante"
    * **Columna Disciplina Relevante:** Identifica la disciplina MÁS RELEVANTE en ese segundo. Debe ser una de:
        * `Striking` - Si el intercambio de golpes es lo más relevante
        * `Grappling` - Si el clinch, derribo o control posicional es lo más relevante
        * `Submission` - Si hay intentos de sumisión o escapes activos
        * `Movement` - Si el posicionamiento, footwork o manejo de distancia es lo más relevante
        * `Mixto` - Si múltiples disciplinas son igualmente relevantes (usar con moderación)

    **CRITERIOS PARA IDENTIFICAR LA DISCIPLINA RELEVANTE:**
    * **Striking:** Cuando hay intercambio activo de golpes (conectados o no), defensas de golpes, o preparación evidente para golpear
    * **Grappling:** Cuando hay clinch, intentos de derribo, control posicional, o trabajo en el suelo sin sumisiones activas
    * **Submission:** Cuando hay intentos activos de sumisión (estrangulaciones, palancas) o escapes defensivos de sumisiones
    * **Movement:** Cuando el posicionamiento, footwork, cambios de ángulo o manejo de distancia es la actividad principal (sin acción de contacto significativa)

    **EJEMPLO DE TABLA CON SAMPLING CADA 1s:**
    | Tiempo (MM:SS) | Peleador | Acción | Disciplina Relevante |
    | :---: | :---: | :--- | :--- |
    | 00:00 | Ambos | Evaluación de distancia inicial | Movement |
    | 00:01 | Ambos | Posicionamiento en centro del octágono | Movement |
    | 00:02 | Peleador A | Jab conectado | Striking |
    | 00:03 | Peleador B | Cross conectado | Striking |
    | 00:04 | Peleador A | Cierre de distancia para derribo | Grappling |
    | 00:05 | Peleador B | Defensa de takedown (sprawl) | Grappling |
    | 00:06 | Peleador A | Control en clinch contra la jaula | Grappling |
    | 00:07 | Peleador A | Intento de estrangulación desde la espalda | Submission |
    | 00:08 | Peleador B | Escape de sumisión | Submission |
    | 00:09 | Ambos | Reset a centro, ajuste de distancia | Movement |
    | 00:10 | Ambos | Sin actividad relevante | Movement |

    **REQUERIMIENTO DE SALIDA:**
    1. Proporciona la tabla de análisis completa con entradas cada 1 segundo, incluyendo las 4 columnas: Tiempo, Peleador, Acción, y Disciplina Relevante.
    2. Al final, proporciona un breve resumen estadístico:
        * Distribución de disciplinas (cuántos segundos fueron Striking, Grappling, Submission, Movement)
        * Momentos clave de la pelea (transiciones importantes entre disciplinas)
        * Peleador dominante por disciplina (si aplica)

    **IMPORTANTE:** Este análisis debe ser objetivo y completo, ya que será la base para que los especialistas (Striking Specialist, Grappling Specialist, Submission Specialist, Movement Specialist) realicen sus análisis detallados posteriormente.
    """
    return prompt

In [8]:
# Generar el prompt del analista general
prompt_general = generate_general_analyst_prompt()

# Ejecutar el análisis general (ground truth)
response_general = client.models.generate_content(
    model="gemini-2.0-flash", 
    contents=[myfile, prompt_general], 
    config={
        "temperature": 0.0,
        "topP": 0.95,
        "seed": 42,
    }
)

# Guardar el análisis general
general_analysis = response_general.text
print(general_analysis)

# Luego, este análisis general se puede pasar a cada especialista
# junto con el video para que hagan su análisis detallado

¡Entendido! Procederé a crear el análisis objetivo y completo segundo a segundo de la pelea, identificando la disciplina más relevante en cada momento y generando la tabla de Markdown solicitada.

Aquí está la tabla de análisis:

| Tiempo (MM:SS) | Peleador | Acción | Disciplina Relevante |
| :---: | :---: | :--- | :--- |
| 00:00 | Ambos | Evaluación de distancia inicial | Movement |
| 00:01 | Ambos | Posicionamiento en centro del octágono | Movement |
| 00:02 | Ambos | Intercambio de distancia | Movement |
| 00:03 | Holloway | Combinación de golpes | Striking |
| 00:04 | Gaethje | Recibe combinación, retrocede | Striking |
| 00:05 | Gaethje | Sangrando por la nariz | Striking |
| 00:06 | Ambos | Posicionamiento | Movement |
| 00:07 | Ambos | Intercambio de distancia | Movement |
| 00:08 | Ambos | Posicionamiento | Movement |
| 00:09 | Ambos | Intercambio de distancia | Movement |
| 00:10 | Ambos | Posicionamiento | Movement |
| 00:11 | Ambos | Intercambio de distancia | Movement |
| 0

In [9]:
def generate_specialist_prompt(role="striking", general_analysis_text=""):
    """
    Genera un prompt para un especialista que recibe el análisis general (ground truth)
    y produce un análisis de fortalezas/debilidades y momentos significativos.
    El prompt es directo y técnico, sin introducciones redundantes sobre la perspectiva del especialista.
    """
    roles = {
        "striking": {
            "name": "Striking Offense/Defense Analyst",
            "desc": "Análisis enfocado en el intercambio de golpes de pie (Boxeo y Muay Thai).",
            "emphasis": "**posicionamiento** (footwork, ángulo de ataque) y la **conexión efectiva** de golpes.",
            "actions": "Jab/Cross Conectado, Patada (al cuerpo, pierna, cabeza), Knockdown, KO/TKO, Esquive Exitoso, Uso de Finta."
        },
        "grappling": {
            "name": "Grappling Analyst",
            "desc": "Análisis enfocado en el clinch, controles contra la jaula, derribos y trabajo de piso.",
            "emphasis": "**control posicional**, **pases de guardia**, **derribos**, y **defensa de derribo**.",
            "actions": "Takedown efectivo, Defensa de takedown, Control en clinch, Pase de guardia, Ground and pound."
        },
        "submission": {
            "name": "Submission Specialist",
            "desc": "Análisis enfocado en intentos de sumisión, transiciones y defensa.",
            "emphasis": "**intentos de sumisión**, **transiciones entre posiciones**, **escapes**.",
            "actions": "Intento de estrangulación, Intento de palanca, Escape de sumisión, Defensa de sumisión."
        },
        "movement": {
            "name": "Movement Specialist",
            "desc": "Análisis enfocado en footwork, posicionamiento, manejo de distancia y ángulos.",
            "emphasis": "**footwork**, **posicionamiento**, **manejo de distancia**, **cambios de ángulo**.",
            "actions": "Cierre de distancia, Ajuste de ángulo, Footwork defensivo, Manejo de espacio, Cambio de guardia."
        }
    }

    role_key = role.lower()
    if role_key not in roles:
        raise ValueError(f"Rol '{role}' no soportado. Opciones: {list(roles.keys())}")

    r = roles[role_key]
    prompt = f"""
        **ROL Y OBJETIVO:**
        Eres un **Asistente de Head Coach de MMA**. Tu especialidad es **{r['name']}**. Has recibido el análisis general (ground truth) de la pelea realizado por el General Analyst. Tu tarea es proporcionar un análisis detallado del rendimiento de los dos peleadores basándote ÚNICAMENTE en este análisis general.

        * **{r['name']}:**
            * **Descripción de la disciplina:** {r['desc']}
            * **Énfasis:** Se centrará en {r['emphasis']}
            * **Acciones Clave a Analizar:** {r['actions']}

        **ANÁLISIS GENERAL PROPORCIONADO:**
        {general_analysis_text}
        
        **INSTRUCCIONES PARA EL ANÁLISIS:**
        * Basándose únicamente en el análisis general proporcionado, identifica todos los momentos donde la disciplina es relevante (según la columna "Disciplina Relevante" o acciones relacionadas con la especialidad).
        * Analiza el rendimiento de cada peleador en detalle, señalando momentos clave, técnicas ejecutadas, errores y transiciones relevantes.
        * Lista y describe los **momentos significativos** de la disciplina con su timestamp (MM:SS).
        * Identifica y describe **fortalezas** de cada peleador en la disciplina, respaldadas con timestamps concretos del análisis general.
        * Identifica y describe **debilidades** de cada peleador en la disciplina, respaldadas con timestamps concretos del análisis general.
        * Todos los hallazgos deben estar respaldados con timestamps (MM:SS) extraídos del análisis general.

        **FORMATO DE SALIDA:**

        ## Análisis de {r['name']}

        ### Momentos Significativos
        [Lista de momentos clave con timestamps (MM:SS) y descripción detallada]

        ### Fortalezas por Peleador
        **Peleador A:**
        - [Fortaleza 1] (timestamp: MM:SS)
        - [Fortaleza 2] (timestamp: MM:SS)
        - ...

        **Peleador B:**
        - [Fortaleza 1] (timestamp: MM:SS)
        - [Fortaleza 2] (timestamp: MM:SS)
        - ...

        ### Debilidades por Peleador
        **Peleador A:**
        - [Debilidad 1] (timestamp: MM:SS)
        - [Debilidad 2] (timestamp: MM:SS)
        - ...

        **Peleador B:**
        - [Debilidad 1] (timestamp: MM:SS)
        - [Debilidad 2] (timestamp: MM:SS)
        - ...

        ### Resumen Ejecutivo
        [Breve resumen directo y técnico de los hallazgos principales. No uses frases como "desde la perspectiva del especialista en...". Expón conclusiones claras y respaldadas.]

        **IMPORTANTE:**
        * Si no hay momentos relevantes en la disciplina dentro del análisis general, indícalo claramente.
        * Sé específico y técnico en las observaciones.
        * Utiliza únicamente los timestamps proporcionados en el análisis general.
        """
    return prompt

In [10]:
specialists = ["striking", "grappling", "submission", "movement"]
specialist_analyses = {}

prompt_specialist = generate_specialist_prompt(
    role=specialists[1],
    general_analysis_text=general_analysis
)

print(prompt_specialist)



        **ROL Y OBJETIVO:**
        Eres un **Asistente de Head Coach de MMA**. Tu especialidad es **Grappling Analyst**. Has recibido el análisis general (ground truth) de la pelea realizado por el General Analyst. Tu tarea es proporcionar un análisis detallado del rendimiento de los dos peleadores basándote ÚNICAMENTE en este análisis general.

        * **Grappling Analyst:**
            * **Descripción de la disciplina:** Análisis enfocado en el clinch, controles contra la jaula, derribos y trabajo de piso.
            * **Énfasis:** Se centrará en **control posicional**, **pases de guardia**, **derribos**, y **defensa de derribo**.
            * **Acciones Clave a Analizar:** Takedown efectivo, Defensa de takedown, Control en clinch, Pase de guardia, Ground and pound.

        **ANÁLISIS GENERAL PROPORCIONADO:**
        ¡Entendido! Procederé a crear el análisis objetivo y completo segundo a segundo de la pelea, identificando la disciplina más relevante en cada momento y generando

In [12]:
specialists = ["striking", "grappling", "submission", "movement"]
specialist_analyses = {}

for specialist_role in specialists:
    print(f"\nAnalizando con {specialist_role} specialist...")
    
    prompt_specialist = generate_specialist_prompt(
        role=specialist_role,
        general_analysis_text=general_analysis
    )
    
    response_specialist = client.models.generate_content(
        model="gemini-2.0-flash",
        contents=[prompt_specialist],
        config={
            "temperature": 0.0,
            "topP": 0.95,
            "seed": 42,
        }
    )
    
    specialist_analyses[specialist_role] = response_specialist.text
    print(f"{specialist_role} analysis completado")


specialist_analyses


Analizando con striking specialist...
striking analysis completado

Analizando con grappling specialist...
grappling analysis completado

Analizando con submission specialist...
submission analysis completado

Analizando con movement specialist...
movement analysis completado


{'striking': '## Análisis de Striking Offense/Defense Analyst\n\n### Momentos Significativos\n\n*   **00:03-00:04:** Holloway conecta una combinación de golpes, forzando a Gaethje a retroceder. Esto indica una superioridad inicial en la velocidad y precisión de Holloway en el striking.\n*   **00:05:** Gaethje sangra por la nariz, resultado directo del striking de Holloway. Esto demuestra la efectividad del ataque de Holloway y la vulnerabilidad de Gaethje.\n*   **00:17-00:18:** Holloway conecta un derechazo, obligando a Gaethje a retroceder nuevamente. Esto refuerza la capacidad de Holloway para conectar golpes significativos.\n*   **00:27-00:29:** Holloway conecta una combinación de golpes, obligando a Gaethje a retroceder repetidamente. Esto sugiere una estrategia de Holloway de presionar con combinaciones.\n*   **00:37-00:38:** Holloway conecta un jab, obligando a Gaethje a retroceder. El jab de Holloway parece ser una herramienta efectiva para controlar la distancia y mantener a Ga

In [28]:
print(specialist_analyses["striking"])

## Análisis de Striking Offense/Defense Analyst

### Momentos Significativos

*   **00:03-00:04:** Holloway conecta una combinación de golpes, forzando a Gaethje a retroceder. Esto indica una superioridad inicial en la velocidad y precisión de Holloway en el striking.
*   **00:05:** Gaethje sangra por la nariz, resultado directo del striking de Holloway. Esto demuestra la efectividad del ataque de Holloway y la vulnerabilidad de Gaethje.
*   **00:17-00:18:** Holloway conecta un derechazo, obligando a Gaethje a retroceder nuevamente. Esto refuerza la capacidad de Holloway para conectar golpes significativos.
*   **00:27-00:29:** Holloway conecta una combinación de golpes, obligando a Gaethje a retroceder repetidamente. Esto sugiere una estrategia de Holloway de presionar con combinaciones.
*   **00:37-00:38:** Holloway conecta un jab, obligando a Gaethje a retroceder. El jab de Holloway parece ser una herramienta efectiva para controlar la distancia y mantener a Gaethje a raya.
*   **00

In [29]:
def generate_head_coach_aggregation_prompt(specialist_analyses):
    """
    Genera un prompt conciso para el Head Coach que agrega todos los análisis.
    """
    # Formatear los análisis de especialistas
    specialist_text = "\n\n".join([
        f"**{role.upper()}:**\n{analysis}" 
        for role, analysis in specialist_analyses.items()
    ])
    
    prompt = f"""
**ROL:** Eres el **Head Coach de MMA**. Sintetiza los análisis de 4 especialistas.

**ANÁLISIS DE ESPECIALISTAS:**
{specialist_text}

**TAREA:**
1. Identifica 3-5 momentos críticos (prioriza por impacto, transiciones, errores costosos)
2. Sintetiza fortalezas/debilidades por peleador (transversal a disciplinas)
3. Genera insights estratégicos (qué funcionó/no funcionó, recomendaciones)

**PRIORIZACIÓN:** Striking > Grappling > Submission > Movement (considera contexto)

**REQUISITOS PARA MOMENTOS CRÍTICOS:**
- **Duración mínima:** Cada momento debe durar **mínimo 3-5 segundos** para ser adecuado para highlights.
- **Formato de timestamps:** Usa rangos de tiempo (ej: "01:23 - 01:28") en lugar de timestamps individuales.
- **Consolidación:** Si encuentras múltiples momentos cortos (1-2 segundos) que están relacionados temáticamente o cronológicamente cercanos (dentro de 10 segundos), consolídalos en un solo momento crítico que incluya el rango completo.
- **Prioridad de consolidación:**
  * Agrupa acciones consecutivas del mismo tipo (ej: combinación de golpes que dura varios segundos)
  * Une transiciones y sus consecuencias inmediatas
  * Combina errores costosos con sus secuelas
- **Evita:** Momentos aislados de 1-2 segundos a menos que sean absolutamente críticos (ej: KO definitivo). Si son importantes pero cortos, busca el contexto circundante para crear un rango más amplio.


**FORMATO:**
## Reporte Ejecutivo

[Timestamp Inicio - Timestamp Fin | Descripción consolidada | Disciplina | Impacto]
Ejemplo: "01:23 - 01:28 | Combinación de golpes que lleva a knockdown | Striking | Alto"

### Perfil por Peleador
**Peleador A:** Fortalezas | Debilidades | Patrones
**Peleador B:** [igual]

### Insights Estratégicos
[Recomendaciones y decisiones clave]
"""
    return prompt

# Para agregar todos los resultados meterle
#Insights Estratégicos
#[Recomendaciones y decisiones clave]

#Disciplina Determinante
#Cuál y por qué]

In [30]:
prompt_head_coach = generate_head_coach_aggregation_prompt(specialist_analyses)
#print(prompt_head_coach)

In [31]:
response_coach = client.models.generate_content(
        model="gemini-2.0-flash",
        contents=[prompt_head_coach],
        config={
            "temperature": 0.0,
            "topP": 0.95,
            "seed": 42,
        }
    )

In [32]:
print(response_coach.text)

## Reporte Ejecutivo

**Momentos Críticos:**

*   **00:51 - 00:53 | Holloway conecta un derechazo que tumba a Gaethje, seguido de un breve intercambio de posicionamiento. | Striking/Movement | Alto**
*   **01:46 - 01:48 | Gaethje conecta dos derechazos consecutivos, mostrando su poder y capacidad de conectar. | Striking | Medio**
*   **02:37 - 02:40 | Holloway conecta una combinación de golpes finalizando con un derechazo que resulta en el KO de Gaethje. | Striking | Alto**

### Perfil por Peleador

**Max Holloway:**

*   **Fortalezas:** Striking versátil y preciso (combinaciones, jab, patadas), control de la distancia, movimiento estratégico, poder de golpeo.
*   **Debilidades:** No se identificaron debilidades significativas en esta pelea.
*   **Patrones:** Presión constante con combinaciones, uso efectivo del jab para controlar la distancia, adaptación constante del movimiento para mantener la ventaja.

**Justin Gaethje:**

*   **Fortalezas:** Poder de golpeo (derechazos), resilienc

In [21]:
from pydantic import BaseModel, Field
from typing import List, Optional
from enum import Enum

class Disciplina(str, Enum):
    STRIKING = "Striking"
    GRAPPLING = "Grappling"
    SUBMISSION = "Submission"
    MOVEMENT = "Movement"

class Impacto(str, Enum):
    MAXIMO = "MÁXIMO"
    SIGNIFICATIVO = "SIGNIFICATIVO"
    MODERADO = "MODERADO"
    BAJO = "BAJO"

class MomentoCritico(BaseModel):
    """Representa un momento crítico identificado en la pelea"""
    timestamp: str = Field(..., description="Timestamp en formato MM:SS o rango MM:SS - MM:SS")
    descripcion: str = Field(..., description="Descripción detallada del momento")
    disciplina: Disciplina = Field(..., description="Disciplina más relevante")
    impacto: Impacto = Field(..., description="Nivel de impacto del momento")

class PerfilPeleador(BaseModel):
    """Perfil de rendimiento de un peleador"""
    nombre: str = Field(..., description="Nombre del peleador")
    fortalezas: List[str] = Field(default_factory=list, description="Lista de fortalezas principales")
    debilidades: List[str] = Field(default_factory=list, description="Lista de debilidades críticas")
    patrones: List[str] = Field(default_factory=list, description="Patrones de comportamiento identificados")

class ReporteEjecutivo(BaseModel):
    """Reporte ejecutivo completo del Head Coach"""
    momentos_criticos: List[MomentoCritico] = Field(
        default_factory=list,
        description="Momentos críticos ordenados por importancia"
    )
    perfil_peleador_a: Optional[PerfilPeleador] = Field(
        None,
        description="Perfil del primer peleador"
    )
    perfil_peleador_b: Optional[PerfilPeleador] = Field(
        None,
        description="Perfil del segundo peleador"
    )
    insights_estrategicos: Optional[List[str]] = Field(
        default_factory=list,
        description="Insights estratégicos y recomendaciones"
    )
    

In [19]:
response_coach = client.models.generate_content(
        model="gemini-2.0-flash",
        contents=[prompt_head_coach],
        config={
            "temperature": 0.0,
            "topP": 0.95,
            "seed": 42,
            "response_mime_type": "application/json",
            "response_json_schema": ReporteEjecutivo.model_json_schema()
        }
    )

KeyboardInterrupt: 

In [42]:
reporteEjec = ReporteEjecutivo.model_validate_json(response_coach.text)

momentos_criticos=[MomentoCritico(timestamp='00:11 - 00:12', descripcion='Chimaev esquiva un overhand de Burns y contraataca con un gancho de izquierda que conecta efectivamente, causando que Burns se tambalee brevemente.', disciplina=<Disciplina.STRIKING: 'Striking'>, impacto=<Impacto.MAXIMO: 'MÁXIMO'>), MomentoCritico(timestamp='00:04 - 00:05', descripcion='Burns lanza un gancho de izquierda; Chimaev evade, cierra la distancia agresivamente y establece el primer clinch contra la jaula, neutralizando la ofensiva de Burns.', disciplina=<Disciplina.MOVEMENT: 'Movement'>, impacto=<Impacto.SIGNIFICATIVO: 'SIGNIFICATIVO'>), MomentoCritico(timestamp='00:09, 00:23, 00:29', descripcion='Burns defiende exitosamente los tres intentos de derribo de Chimaev, demostrando una sólida base y defensa de derribo, impidiendo que la pelea vaya al suelo.', disciplina=<Disciplina.GRAPPLING: 'Grappling'>, impacto=<Impacto.SIGNIFICATIVO: 'SIGNIFICATIVO'>), MomentoCritico(timestamp='00:10, 00:15, 00:23, 00:30

In [43]:
print(reporteEjec.model_dump_json(indent=2, ensure_ascii=False))

{
  "momentos_criticos": [
    {
      "timestamp": "00:11 - 00:12",
      "descripcion": "Chimaev esquiva un overhand de Burns y contraataca con un gancho de izquierda que conecta efectivamente, causando que Burns se tambalee brevemente.",
      "disciplina": "Striking",
      "impacto": "MÁXIMO"
    },
    {
      "timestamp": "00:04 - 00:05",
      "descripcion": "Burns lanza un gancho de izquierda; Chimaev evade, cierra la distancia agresivamente y establece el primer clinch contra la jaula, neutralizando la ofensiva de Burns.",
      "disciplina": "Movement",
      "impacto": "SIGNIFICATIVO"
    },
    {
      "timestamp": "00:09, 00:23, 00:29",
      "descripcion": "Burns defiende exitosamente los tres intentos de derribo de Chimaev, demostrando una sólida base y defensa de derribo, impidiendo que la pelea vaya al suelo.",
      "disciplina": "Grappling",
      "impacto": "SIGNIFICATIVO"
    },
    {
      "timestamp": "00:10, 00:15, 00:23, 00:30",
      "descripcion": "Burns log